In [1]:
using SymPy

In [2]:
@vars x
expr = 1/(1+x^2)

  1   
------
 2    
x  + 1

In [3]:
@which display(expr)

display(x) in Base.Multimedia at multimedia.jl:324

In [4]:
d = Base.Multimedia.displays[end]

IJulia.InlineDisplay()

In [5]:
@which display(d, expr)

display(d::IJulia.InlineDisplay, x) in IJulia at D:\.julia\packages\IJulia\AQu2H\src\inline.jl:95

In [6]:
IJulia.display_dict(x)

Dict{String, Union{String, JSON.Writer.JSONText}} with 2 entries:
  "text/plain" => "x"
  "text/latex" => "\$x\$"

In [7]:
@which IJulia.display_dict(x)

display_dict(x) in IJulia at D:\.julia\packages\IJulia\AQu2H\src\display.jl:97

In [8]:
IJulia.ijulia_mime_types

5-element Vector{Union{MIME, AbstractVector{MIME}}}:
 MIME type text/plain
 MIME type image/svg+xml
 MIME[MIME type image/png, MIME type image/jpeg]
 MIME[MIME type text/markdown, MIME type text/html]
 MIME type text/latex

In [9]:
m = IJulia.ijulia_mime_types[end]

MIME type text/latex

In [10]:
IJulia.display_mimestring(m, x)

(MIME type text/latex, "\$x\$")

In [11]:
@which IJulia.display_mimestring(m, x)

display_mimestring(m::MIME, x) in IJulia at D:\.julia\packages\IJulia\AQu2H\src\display.jl:67

In [12]:
IJulia.limitstringmime(m, x)

"\$x\$"

In [13]:
@which IJulia.limitstringmime(m, x)

limitstringmime(mime::MIME, x) in IJulia at D:\.julia\packages\IJulia\AQu2H\src\inline.jl:37

In [14]:
show(stdout, m, expr)

$\frac{1}{x^{2} + 1}$

In [15]:
@which show(stdout, m, expr)

show(io::IO, ::MIME{Symbol("text/latex")}, x::SymPy.SymbolicObject) in SymPy at D:\.julia\packages\SymPy\5GXQf\src\types.jl:87

In [16]:
sympy.latex(x, mode="inline",fold_short_frac=false)

"\$x\$"

In [17]:
?sympy.latex

Convert the given expression to LaTeX string representation.

    Parameters
    full_prec: boolean, optional
        If set to True, a floating point number is printed with full precision.
    fold_frac_powers : boolean, optional
        Emit ``^{p/q}`` instead of ``^{\frac{p}{q}}`` for fractional powers.
    fold_func_brackets : boolean, optional
        Fold function brackets where applicable.
    fold_short_frac : boolean, optional
        Emit ``p / q`` instead of ``\frac{p}{q}`` when the denominator is
        simple enough (at most two terms and no powers). The default value is
        ``True`` for inline mode, ``False`` otherwise.
    inv_trig_style : string, optional
        How inverse trig functions should be displayed. Can be one of
        ``abbreviated``, ``full``, or ``power``. Defaults to ``abbreviated``.
    itex : boolean, optional
        Specifies if itex-specific syntax is used, including emitting
        ``$$...$$``.
    ln_notation : boolean, optional
        If 

In [18]:
@which sympy.latex(x, mode="inline",fold_short_frac=false)

(::Core.var"#Any##kw")(::Any, o::PyCall.PyObject, args...) in PyCall at D:\.julia\packages\PyCall\7a7w0\src\pyfncall.jl:86

In [19]:
sympy.latex(x, mode="equation", itex=true, fold_short_frac=false)

"\$\$x\$\$"

In [20]:
sympy.latex(x, mode="equation", fold_short_frac=false)

"\\begin{equation}x\\end{equation}"

In [21]:
# original: https://github.com/JuliaPy/SymPy.jl/blob/master/src/types.jl#L87
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::SymbolicObject)
    print(io, sympy.latex(x, mode="inline",fold_short_frac=false)) # plain? equation*?
#    #print(io, as_markdown(sympy.latex(x, mode="equation*")))
#    #print(io, as_markdown(sympy.latex(x, mode="plain",fold_short_frac=false))) # inline?
end

In [22]:
@eval SymPy Base.show(io::IO, ::MIME"text/latex", x::SymbolicObject) = print(io, sympy.latex(x, mode="equation*"))

In [23]:
expr

  1   
------
 2    
x  + 1

In [24]:
y = [expr, expr]

2-element Vector{Sym}:
 1/(x^2 + 1)
 1/(x^2 + 1)

In [25]:
Base.show(stdout, m, y)

$\left[ \begin{array}{r}\frac{1}{x^{2} + 1}\\\frac{1}{x^{2} + 1}\end{array} \right]$


In [26]:
using Markdown

In [27]:
Markdown.parse("``$expr``")

$1/(x^2 + 1)$


In [28]:
@which Markdown.parse("``$expr``")

parse(markdown::AbstractString; flavor) in Markdown at D:\Julia-1.7.2\share\julia\stdlib\v1.7\Markdown\src\Markdown.jl:31

In [29]:
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::AbstractArray{Sym})
    function toeqnarray(x::Vector{Sym})
        a = join([sympy.latex(x[i]) for i in 1:length(x)], "\\\\")
        """\\left[ \\begin{array}{r}$a\\end{array} \\right]"""
#        "\\begin{bmatrix}$a\\end{bmatrix}"
    end
    function toeqnarray(x::AbstractArray{Sym,2})
        sz = size(x)
        a = join([join(map(sympy.latex, x[i,:]), "&") for i in 1:sz[1]], "\\\\")
        "\\left[ \\begin{array}{" * repeat("r",sz[2]) * "}" * a * "\\end{array}\\right]"
#        "\\begin{bmatrix}$a\\end{bmatrix}"
    end
    print(io, as_markdown(toeqnarray(x)))
end

In [30]:
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::AbstractArray{Sym})
    function toeqnarray(x::Vector{Sym})
        a = join(["\\displaystyle" * sympy.latex(x[i]) for i in 1:length(x)], "\\\\")
        """\\left[ \\begin{array}{r}$a\\end{array} \\right]"""
#        "\\begin{bmatrix}$a\\end{bmatrix}"
    end
    function toeqnarray(x::AbstractArray{Sym,2})
        sz = size(x)
        a = join([join("\\displaystyle" .* map(sympy.latex, x[i,:]), "&") for i in 1:sz[1]], "\\\\")
        "\\left[ \\begin{array}{" * repeat("r",sz[2]) * "}" * a * "\\end{array}\\right]"
#        "\\begin{bmatrix}$a\\end{bmatrix}"
    end
    print(io, as_markdown(toeqnarray(x)))
end

In [31]:
y

2-element Vector{Sym}:
 1/(x^2 + 1)
 1/(x^2 + 1)

In [32]:
z = [y y]

2×2 Matrix{Sym}:
 1/(x^2 + 1)  1/(x^2 + 1)
 1/(x^2 + 1)  1/(x^2 + 1)